In [ ]:
# double pendulum 2D

%matplotlib notebook

import numpy as np
import sympy as sp
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from matplotlib import animation
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import PillowWriter
import matplotlib.patheffects as path_effects


# define symbols symbolically 
m1  = sp.symbols('m1')
m2  = sp.symbols('m2')
L1  = sp.symbols('L1')
L2  = sp.symbols('L2')
g  = sp.symbols('g')
t  = sp.symbols('t')

# define thetas as symbolic functions of time
# theta 1 is the angle between the string of the first pendulum
# and the vertical
# theta 2 is the angle between the string of the second pendulum
# and the vertical
theta1 = sp.Function('theta_1')(t)
theta2 = sp.Function('theta_2')(t)

# now find first derivative of theta's symbolically
# this is derivative of theta 1 with respect to time 
# and derivative of theta 2 with respect to time
dtheta1 = theta1.diff(t)
dtheta2 = theta2.diff(t)

# now find second derivatives of theta's
# by taking the derivative of the first derivative,
# again with respect to time
d2theta1 = dtheta1.diff(t)
d2theta2 = dtheta2.diff(t)

# now define the x and y position of the pendulum bobs
# as dependent on theta
# these equations given in info about double pendulums
# and can also be derivated from a free body diagram
x1 = L1*sp.sin(theta1)
x2 = L1*sp.sin(theta1)+L2*sp.sin(theta2)

y1 = -L1*sp.cos(theta1)
y2 = -L1*sp.cos(theta1)-L2*sp.cos(theta2)

# now find derivatives of positions to get velocities of the bob's
dx1 = x1.diff(t)
dx2 = x2.diff(t)

dy1 = y1.diff(t)
dy2 = y2.diff(t)
# we can display these symbolically and it looks nice

# the Lagrangian is L = T-V - T is kinetic energy
# and V is potential energy

# Kinetic
# kinetric energy is 1/2*m*v^2
# need to find kinetic energy of each bob - where the velcocity
# of each bob is found by adding the components
kineticEnergy1 = 1/2 * m1 * (dx1**2 + dy1**2)
kineticEnergy2 = 1/2 * m2 * (dx2**2 + dy2**2)
kineticEnergy = kineticEnergy1 + kineticEnergy2
# Potential 
# potential energy is m*g*h - where h is y
potentialEnergy1 = m1*g*y1
potentialEnergy2 = m2*g*y2
potentialEnergy = potentialEnergy1 + potentialEnergy2
# Lagrangian
Lagrangian = kineticEnergy-potentialEnergy

# equations of motion are Euler-Lagrange equations
# d/dt(dL/d dtheta) - dL/dtheta = 0 for theta = theta1, theta2
ELangrange1 = (Lagrangian.diff(dtheta1)).diff(t)- Lagrangian.diff(theta1)
ELangrange2 = (Lagrangian.diff(dtheta2)).diff(t)-Lagrangian.diff(theta2)

#these are both equal to zero

# now we solve them to get 2 second order differential equations
solution = sp.solve([ELangrange1, ELangrange2], (d2theta1, d2theta2))

# now to solve the system of equations we break it up into 4 first order
# differnetial equations
# so let u1 = dtheta1 and u2 = dtheta2
# then we have
# dtheta1 = u1, dtheta2 = u2, du1, and du2
# we will solve them numerically
# so we need numerical functions, not symbolic
# use labdify to convert to numerical - lambda function

# first input is variable, second is expression
dtheta1Num = sp.lambdify(dtheta1, dtheta1)
dtheta2Num = sp.lambdify(dtheta2, dtheta2)
du1Num = sp.lambdify((t,g,m1,m2,L1,L2,theta1,theta2,dtheta1,dtheta2),solution[d2theta1])
du2Num = sp.lambdify((t,g,m1,m2,L1,L2,theta1,theta2,dtheta1,dtheta2),solution[d2theta2])

# also make numerical x and y functions that we will use later
x1Num = sp.lambdify((theta1, theta2, L1, L2),x1)
x2Num = sp.lambdify((theta1, theta2, L1, L2),x2)

y1Num = sp.lambdify((theta1, theta2, L1, L2),y1)
y2Num = sp.lambdify((theta1, theta2, L1, L2),y2)

# solve system using solve_ivp
#needs to be in form dW/dt = f(t,w)
def returnsdW(W,t, g, m1, m2, L1, L2):
    theta1 , u1, theta2, u2 = W
    
    
    dtheta1R = dtheta1Num(u1)
    du1R = du1Num(t, g, m1, m2, L1, L2, theta1, theta2, u1, u2)
    dtheta2R = dtheta2Num(u2)
    du2R = du2Num(t, g, m1, m2, L1, L2, theta1, theta2, u1, u2)
    
    return [dtheta1R, du1R, dtheta2R, du2R]

# time points
t = np.linspace(0, 50, 501)

g = 9.81
# user input for mass and length
m1= int(input("Enter mass 1:\n"))
m2= int(input("Enter mass 2:\n"))
L1 = int(input("Enter length 1:\n"))
L2 = int(input("Enter length 2:\n"))

y0 = [1,0,-1,0]

# solve ODE
z = odeint(returnsdW, y0, t=t, args=(g,m1,m2,L1,L2))

# get columns that corespond to functions
theta1 = z[:,0]
u1 = z[:,1]
theta2 = z[:,2]
u2 = z[:,3]

# now we have to get the coordinates for animation
def getCoords(theta1, theta2, L1, L2):
    return (x1Num(theta1, theta2, L1, L2), y1Num(theta1, theta2, L1, L2),
            x2Num(theta1, theta2, L1, L2), y2Num(theta1, theta2, L1, L2)
    )

# call getCoords
x1, y1, x2, y2 = getCoords(theta1, theta2, L1, L2)


def animate(i):
    pic.set_data([0, x1[i], x2[i]], [0, y1[i], y2[i]])
    
    
fig, ax = plt.subplots(1,1, figsize=(8,8))
pic, = plt.plot([], [], 'ko-', lw=3, markersize=15, path_effects = [path_effects.SimpleLineShadow(),
                       path_effects.Normal()])
ax.set_ylim(-(L1+L2)*2,(L1+L2)*2)
ax.set_xlim(-(L1+L2)*2,(L1+L2)*2)
ani = animation.FuncAnimation(fig, animate, frames=500, interval=50)
ani.save('pen.gif',writer='pillow',fps=25)


In [ ]:
# 3D Double Pendulum Animation
#Requires an older version of sympy to run smp.solve

!pip install sympy==1.6.2
import numpy as np
import sympy as smp
from scipy.integrate import odeint
from sympy.solvers.solveset import linsolve, nonlinsolve
import matplotlib.pyplot as plt

#We will define these as variables in sympy and give them values later
#t is time and g is gravitational constant and theta and phi are azimuthal angles
t, g = smp.symbols('t, g')
mass1, mass2 = smp.symbols('mass1, mass2')
Len1, Len2 = smp.symbols('Len1, Len2')

ϕ1, θ1, ϕ2, θ2 = smp.symbols(r'\theta_1, \theta_2, \phi_1, \phi_2', cls=smp.Function) #\theta and hit tab

#Write each as a function of time
θ1 = θ1(t)
θ2 = θ2(t)
ϕ1 = ϕ1(t)
ϕ2 = ϕ2(t)

#first derivatives with respect to time

θ1_d = smp.diff(θ1, t)
θ2_d = smp.diff(θ2, t)
ϕ1_d = smp.diff(ϕ1, t)
ϕ2_d = smp.diff(ϕ2, t)

#2nd derivatives with respect to time
ϕ1_dd = smp.diff(ϕ1_d, t)
ϕ2_dd = smp.diff(ϕ2_d, t)
θ1_dd = smp.diff(θ1_d, t)
θ2_dd = smp.diff(θ2_d, t)

#define xyz coordinates in terms of parameters
x1 = Len1*smp.sin(θ1)*smp.cos(ϕ1)
y1 = Len1*smp.sin(θ1)*smp.sin(ϕ1)
z1 = -Len1*smp.cos(θ1)
x2 = x1 + Len2*smp.sin(θ2)*smp.cos(ϕ2)
y2 = y1 + Len2*smp.sin(θ2)*smp.sin(ϕ2)
z2 = z1 - Len2*smp.cos(θ2)

#Numerical functions
x1_eqn = smp.lambdify((θ1, θ2, ϕ1, ϕ2, Len1, Len2), x1)
y1_eqn = smp.lambdify((θ1, θ2, ϕ1, ϕ2, Len1, Len2), y1)
z1_eqn = smp.lambdify((θ1, θ2, ϕ1, ϕ2, Len1, Len2), z1)
x2_eqn = smp.lambdify((θ1, θ2, ϕ1, ϕ2, Len1, Len2), x2)
y2_eqn = smp.lambdify((θ1, θ2, ϕ1, ϕ2, Len1, Len2), y2)
z2_eqn = smp.lambdify((θ1, θ2, ϕ1, ϕ2, Len1, Len2), z2)

#Use numerical functions to define potential and kinetic energy to get Lagrangian equations
K1 = 1/2 * mass1 * (smp.diff(x1, t)**2 + smp.diff(y1, t)**2 + smp.diff(z1, t)**2)
K2 = 1/2 * mass2 * (smp.diff(x2, t)**2 + smp.diff(y2, t)**2 + smp.diff(z2, t)**2)
T = K1 + K2

#Potential energy
P1 = mass1*g*z1
P2 = mass2*g*z2
P = P1 + P2

#Lagrangian
L = T - P

#Building Lagrangian equations, as each is set each equal to 0
Lg_eqn1 = smp.diff(L, θ1) - smp.diff(smp.diff(L, θ1_d), t).simplify()
Lg_eqn2 = smp.diff(L, θ2) - smp.diff(smp.diff(L, θ2_d), t).simplify()
Lg_eqn3 = smp.diff(L, ϕ1) - smp.diff(smp.diff(L, ϕ1_d), t).simplify()
Lg_eqn4 = smp.diff(L, ϕ2) - smp.diff(smp.diff(L, ϕ2_d), t).simplify()

#Solves Lagrange equations
sols = smp.solve([Lg_eqn1, Lg_eqn2, Lg_eqn3, Lg_eqn4], (θ1_dd, θ2_dd, ϕ1_dd, ϕ2_dd),
                 simplify=False, rational=False)
#These are second order ODE's, but python can only solve first order, so we will rewrite these as though they are first order ODE's
#smp.solve sets each equal to zero

#We define new dummy variables so that we can convert these into 8 separate
# first order ODE's and then convert them to numerical functions with smp.lambdify
#to be solved

da1dt = smp.lambdify((t,g,mass1,mass2,Len1,Len2,θ1,θ2,θ1_d,θ2_d,ϕ1,ϕ2,ϕ1_d,ϕ2_d), sols[θ1_dd])
da2dt = smp.lambdify((t,g,mass1,mass2,Len1,Len2,θ1,θ2,θ1_d,θ2_d,ϕ1,ϕ2,ϕ1_d,ϕ2_d), sols[θ2_dd])
dθ1dt = smp.lambdify(θ1_d, θ1_d)
dθ2dt = smp.lambdify(θ2_d, θ2_d)

db1dt = smp.lambdify((t,g,mass1,mass2,Len1,Len2,θ1,θ2,θ1_d,θ2_d,ϕ1,ϕ2,ϕ1_d,ϕ2_d), sols[ϕ1_dd])
db2dt = smp.lambdify((t,g,mass1,mass2,Len1,Len2,θ1,θ2,θ1_d,θ2_d,ϕ1,ϕ2,ϕ1_d,ϕ2_d), sols[ϕ2_dd])
dϕ1dt = smp.lambdify(ϕ1_d, ϕ1_d)
dϕ2dt = smp.lambdify(ϕ2_d, ϕ2_d)

#We define a vector C as a function of time using our variables and returns dC/dt

def dCdt(C, t, g, mass1, mass2, Len1, Len2):
    θ1, θ2, ϕ1, ϕ2, a1, a2, b1, b2 = C
    return [
        dθ1dt(a1),
        dθ2dt(a2),
        dϕ1dt(b1),
        dϕ2dt(b2),
        da1dt(t, g, mass1, mass2, Len1, Len2, θ1, θ2, a1, a2, ϕ1, ϕ2, b1, b2),
        da2dt(t, g, mass1, mass2, Len1, Len2, θ1, θ2, a1, a2, ϕ1, ϕ2, b1, b2),
        db1dt(t, g, mass1, mass2, Len1, Len2, θ1, θ2, a1, a2, ϕ1 , ϕ2, b1, b2),
        db2dt(t, g, mass1, mass2, Len1, Len2, θ1 , θ2, a1, a2, ϕ1, ϕ2, b1, b2),
    ]

#We define our missing variables for time, gravity etc. and use odeint to solve system
# of ODE's. linspace and ans.shape used for animation info.

t = np.linspace(0, 40, 1001)
g = 9.81
mass1 = 2
mass2 = 2
Len1 = 2
Len2 = 2
ans = odeint(dCdt, y0=[np.pi/3, np.pi/3, 0, -np.pi, 3, 10, -2, -10], t=t, args=(g,mass1,mass2,Len1,Len2))

ans.shape

θ1 = ans.T[0]
θ2 = ans.T[1]
ϕ1 = ans.T[2]
ϕ2 = ans.T[3]

#positions are in polar form and we use our previously defined functions to convert to 
#cartesian coordinates
def position(θ1, θ2, ϕ1, ϕ2, Len1, Len2):
    return (x1_eqn(θ1, θ2, ϕ1, ϕ2, Len1, Len2),
            y1_eqn(θ1, θ2, ϕ1, ϕ2, Len1, Len2),
            z1_eqn(θ1, θ2, ϕ1, ϕ2, Len1, Len2),
            x2_eqn(θ1, θ2, ϕ1, ϕ2, Len1, Len2),
            y2_eqn(θ1, θ2, ϕ1, ϕ2, Len1, Len2),
            z2_eqn(θ1, θ2, ϕ1, ϕ2, Len1, Len2))


x1, y1, z1, x2, y2, z2 = position(ans.T[0], ans.T[1], ans.T[2], ans.T[3], Len1, Len2)

#save as a numpy array to feed into animation
np.save('data_3D_pendulum', np.array([x1,y1,z1,x2,y2,z2]))

#Animation
import vpython
from vpython import *
import numpy as np

#call shapes, give starting positions
#Note that y is the vertical axis in Vpython
x1, y1, z1, x2, y2, z2 = np.load('data_3D_pendulum.npy')
ball = vpython.sphere(color = color.white, radius = 0.1)
pend1 = vpython.sphere(color = color.yellow, radius = 0.3, make_trail=True, retain=10)
pend2 = vpython.sphere(color = color.blue, radius = 0.3, make_trail=True, retain=10)
bar1 = cylinder(pos=vector(0,0,0),axis=vector(0,0,0), radius=0.05)
bar2 = cylinder(pos=vector(0,0,0),axis=vector(0,0,0), radius=0.05)
ground  = box(pos=vector(0,-4.25,0),axis=vector(1,0,0),
            size=vector(10,0.5,10), color= color.yellow)
shadow1 = cylinder(pos=vector(0,-3.99,0),axis=vector(0,-0.1,0), radius=0.5, color=color.green)
shadow2 = cylinder(pos=vector(0,-3.99,0),axis=vector(0,-0.1,0), radius=0.5, color=color.green)

#Use while loop, rate of 20 works best
print('Start')
i = 0
while True:
    rate(20)
    i = i + 1
    i = i % len(x1)
    ball.pos = vector(0,0,0)
    pend1.pos = vector(x1[i], z1[i], y1[i])
    pend2.pos = vector(x2[i], z2[i], y2[i])
    bar1.axis = vector(x1[i], z1[i], y1[i])
    bar2.pos = vector(x1[i], z1[i], y1[i])
    bar2.axis = vector(x2[i]-x1[i], z2[i]-z1[i], y2[i]-y1[i])
    shadow1.pos = vector(x1[i], -3.99, y1[i])
    shadow2.pos = vector(x2[i], -3.99, y2[i])
